In [1]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 28.3 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
     |████████████████████████████████| 636 kB 65.9 MB/s 
     |████████████████████████████████| 895 kB 59.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Processing the data

pytorch로 batch 1에 sequence classifier train하는 방법이다.

In [2]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# This is new
batch["labels"] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

MRPC(Microsoft Research Paraphrase Corpus) 데이터 세트를 예로 사용할 것이다!

## Loading a dataset from the Hub

hub에서 모델 뿐만 아니라 데이터셋도 포함되어있다!

MRPC는 GLUE benchmark를 구성하는 10개 dataset 중 한 개이다!

10개의 다른 text classification task에서 성능 측정하는데 사용된다!!

dataset load(MRPC)

In [7]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Downloading: 0.00B [00:00, ?B/s]

Downloading: 0.00B [00:00, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

MRPC는 train set, validation set, test set을 포함한 DatasetDict가 존재한다.

In [8]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'idx': 0,
 'label': 1,
 'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'}

In [9]:
raw_train_dataset.features # dataset feature들 보여준다!

{'idx': Value(dtype='int32', id=None),
 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], names_file=None, id=None),
 'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None)}

## Preprocessing a dataset

In [10]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])

In [12]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

toekn_type_ids가 sentence 번호를 알려준다!

In [14]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 'sentence',
 '.',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 'one',
 '.',
 '[SEP]']

[CLS] sentence1 [SEP] sentence2 [SEP]로 input이 들어간다.

[CLS] sentence1 [SEP]는 token type ids 0, sentence2 [SEP] token type is 1로 표시된다.

다른 checkpoint(ex. DistillBERT)를 사용하면 token_type_ids가 없을 수 있다.

BERT는 마스킹 언어 모델링 뿐만 아니라 문장 간의 관계 task도 있다.

**NSP로 문장의 쌍을 제공하고 두 번째 문장이 첫 번째 문장을 따르는지 예측을 요청받는다.**

**non-trivial하게, 원래 document에서 뽑은 문장 반은 서로를 follow하고, 나머지 반은 2개의 다른 document로부터 두 문장을 가져온다!**

In [15]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)

In [16]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

Dataset.map: 데이터 세트로 유지하기 위해서 사용

batched=True할 시, 토큰화 속도를 크게 높일 수 있다!

**해당 batch마다 padding을 하면 확실히 효율적이다!**

batched=Ture로 한 번에 Dataset의 여러 요소에 적용할 수 있다. => 더 빨리 전처리가 가능하다!

In [17]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 408
    })
    test: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 1725
    })
})

num_proc argument를 전달하여 Dataset.map으로 preprocess 함수를 적용할 때도 multiprocessing을 할 수 있다.

우선, Tokenizer library가 여러 thread를 사용하여 sample을 더 빨리 토큰화하므로 사용했다. 만일 이 tokenizer를 사용하지 않을 경우 처리 속도가 빨라질 수도 있다.

## Dynamin padding

Pytorch에서 batch 내에 sample을 조립하는 기능을 collate함수라고 한다.

collate는 DataLoader의 인수이다.

**batch에 따라 패딩하여 훈련 속도 향상을 원했지만 TPU에서 훈련 시에는 문제 발생**

TPU는 고정 모양을 선호!! -> 그래서 추가 패딩이 필요!

그래서, batch 처리 시, 올바른 양의 패딩을 적용하는 collate를 정의해야 하는데 **Transformers DataCollator**로 이러한 기능 제공!

In [18]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

필요로 하지 않는 문저열은 idx, sentence1, sentence2를 제거하고 각 batch에 있는 항목의 길이를 보자!

In [19]:
samples = tokenized_datasets["train"][:8]
samples = {
    k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]
}
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

위의 결과로 봤듯이, 32~67 길이로 다양한 길이의 샘플이 보여진다!

dynamic padding이 없다면, 모든 샘플을 최대 길이로 padding해아한다!

**data_collator로 batch에 dynamic하게 padding되어 있는지 확인을 다시 할 수 있다!**

In [20]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'attention_mask': torch.Size([8, 67]),
 'input_ids': torch.Size([8, 67]),
 'labels': torch.Size([8]),
 'token_type_ids': torch.Size([8, 67])}